In [1]:
import sys
sys.path.append('../LegalBot')

from RAG_v1 import RAG_Bot

import numpy as np
import pandas as pd
from tqdm import tqdm

from ragas.metrics import faithfulness, answer_relevancy
from nltk.translate.bleu_score import sentence_bleu
from ragas import evaluate
from datasets import Dataset 

In [2]:
collection_names = ['Uk', 'Wales', 'NothernIreland', 'Scotland']
bot = RAG_Bot(collection_names=collection_names, text_splitter='SpaCy', embedding_model="SentenceTransformers")

Cluster_URL: https://gwqaighsqp6gtrk8kwmnmg.c0.us-west3.gcp.weaviate.cloud, Cluster_API: 2xKmTTGUawzXYItFrMR9JgTjY2oRkfzHFLWN


I0000 00:00:1726349662.555374    6820 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


Creating 4 Weaviate Clusters - Option 3
Country.capitalize(): Uk
The collection: Uk already exists in the Weaviate Cluster
Country.capitalize(): Wales
The collection: Wales already exists in the Weaviate Cluster
Country.capitalize(): Nothernireland
The collection: NothernIreland already exists in the Weaviate Cluster
Country.capitalize(): Scotland
The collection: Scotland already exists in the Weaviate Cluster


In [3]:
# df = pd.read_csv('./Benchmark-Data.csv')
# df

In [4]:
# responses = []
# for idxrow, row in tqdm(df.iterrows()):
#     country = row['Country']
#     prompt = row['Question']
#     response = bot.query(query=country+' '+prompt, k=1, search_type='Hybrid',)
#     responses.append(response)

In [5]:
# df['Response'] = responses
# df.to_csv('./RAG_Bot_Responses.csv', index=False)
# df

# Now Getting the Metric Values

In [6]:
df = pd.read_csv('./RAG_Bot_Responses.csv')
df

,Country,Question,Actual Answer,Response
0,uk,In the context of the Environmental Protection...,The Environmental Land Management (ELM) scheme...,The provided text focuses on environmental reg...
1,uk,"Under the Employment Rights Act 1996, as amend...","The Employment Rights Act 1996, as amended by ...",Under the Employment Rights Act 1996 as amende...
2,uk,"In light of the Data Protection Act 2018, inco...",The situation described involves potential bre...,"Under the UK Data Protection Act 2018 (DPA), t..."
3,uk,Following the passage of the Renters' Reform B...,"The Renters' Reform Bill, expected to receive ...",The passage you provided focuses on the Leaseh...
4,uk,"Under the Building Safety Act 2022, a construc...",The Building Safety Act 2022 introduces string...,The Building Safety Act 2022 specifically addr...
5,Wales,In light of the Welsh Language (Wales) Measure...,The Welsh Language (Wales) Measure 2011 establ...,Public bodies in Wales are required to accommo...
6,Wales,Considering the Environment (Wales) Act 2016 a...,The Environment (Wales) Act 2016 and the Agric...,"The provided text focuses on the ""Agriculture ..."
7,Wales,"Under the Renting Homes (Wales) Act 2016, as a...","The Renting Homes (Wales) Act 2016, amended by...",The Renting Homes (Wales) Act 2016 introduced ...
8,Wales,Following the implementation of the Additional...,The Additional Learning Needs and Education Tr...,The Additional Learning Needs and Education Tr...
9,Wales,In the context of the Planning (Wales) Act 201...,The Planning (Wales) Act 2015 and the Well-bei...,"The Planning (Wales) Act 2015, along with the ..."


In [7]:
Faithfulness_List = []
Answer_Relevancy_List = []
Blue_Score_List = []
for idxdata, data in df.iterrows():
    data_sample = {
        'question': [data['Question']],
        'answer': [data['Actual Answer']],
        'contexts': [[data['Response']]],
    }
    dataset = Dataset.from_dict(data_sample)
    score = evaluate(dataset,metrics=[faithfulness, answer_relevancy])
    score.to_pandas()
    Faithfulness_List.append(score['faithfulness'])
    Answer_Relevancy_List.append(score['answer_relevancy'])
    blue_score = sentence_bleu([data['Actual Answer'].split()], data['Response'].split())
    Blue_Score_List.append(blue_score)
    
    print(f'Blue Score: {blue_score} - Scores: {score}')
    
print(f'\nMean Value of Faithfulness: {np.mean(Faithfulness_List)}')
print(f'Mean Value of Answer Relevancy: {np.mean(Answer_Relevancy_List)}')
print(f'Mean Value of Blue Score: {np.mean(Blue_Score_List)}')

Evaluating: 100%|██████████| 2/2 [00:19<00:00,  9.81s/it]


Blue Score: 0.010333151388370753 - Scores: {'faithfulness': 0.2857, 'answer_relevancy': 0.0000}


Evaluating: 100%|██████████| 2/2 [00:14<00:00,  7.06s/it]


Blue Score: 0.08464664274532313 - Scores: {'faithfulness': 0.5000, 'answer_relevancy': 0.9211}


Evaluating: 100%|██████████| 2/2 [00:13<00:00,  6.54s/it]


...

Blue Score: 0.044615446157330506 - Scores: {'faithfulness': 0.3043, 'answer_relevancy': 0.9314}


Evaluating: 100%|██████████| 2/2 [00:11<00:00,  5.76s/it]


Blue Score: 0.10358131229281371 - Scores: {'faithfulness': 0.6471, 'answer_relevancy': 0.9079}


Evaluating: 100%|██████████| 2/2 [00:20<00:00, 10.09s/it]


Blue Score: 0.11071808554112603 - Scores: {'faithfulness': 0.6800, 'answer_relevancy': 0.9272}

Mean Value of Faithfulness: 0.35628920876651055
Mean Value of Answer Relevancy: 0.738605332204463
Mean Value of Blue Score: 0.07685320315024469
